# HW4. Indexes & Query Processing & Normalization

## Objectives

In this assignment, you will review what you have learned in the Normalization, Views and Indexes, and Query Processing Modules. In particular, you will practice:

- How indexing can change query processing
- How indexing changes query performance
- How B-Trees store records
- Query processing and optimization
- How to construct BCNF and 3NF decomposition

## Q1 (10 points): Indexing

In this question, you will be asked to select suitable indexes to speed up query performance and examine the query plan of a SQL query.

We are going to use a new database called flights.db. In the database, there is a single table, called FLIGHTS. The following shows its schema:

    FLIGHTS(fid, year, month_id, day_of_month, day_of_week_id, 
            carrier_id, flight_num, origin_city, origin_state, 
            dest_city, dest_state, departure_delay, taxi_out, 
            arrival_delay, canceled, actual_time, distance)

Note that this task only needs to use four attributes: `carrier_id`, `origin_city`, `actual_time`, and `dest_city`.

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///flight.db

'Connected: @flight.db'

Consider the following queries:

```sqlite
(a)  SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Seattle WA' AND actual_time <= 180;
```


```sqlite
(b)  SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Gunnison CO' AND actual_time <= 180;
```


```sqlite
(c)  SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Seattle WA' AND actual_time <= 30;
```

Choose one single simple index (index on one attribute) that is most likely to speed up all three queries. Write down the `CREATE INDEX` statement and explain why you chose that index below.

Q1.1. (1 point) What is the CREATE INDEX statement?

In [4]:
%%sql
CREATE INDEX idx_origin_city ON FLIGHTS(origin_city);

 * sqlite:///flight.db
(sqlite3.OperationalError) index idx_origin_city already exists
[SQL: CREATE INDEX idx_origin_city ON FLIGHTS(origin_city);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Q1.2. (1 point) Why did you choose the index? 

I chose the index using the attribute actual_time, because the data is then optimizaed for any querey later using this attribure. 

Open a command line shell and start the sqlite program. Connect to the provided flights.db, and check whether the FLIGHTS table has the index that you indicate above. If not, add this index to the FLIGHTS table. 

Q1.3. (0.5 point) Does the FLIGHTS table has the index that you indicate above?

In [5]:
%%sql
/*
FLIGHTS table does in fact have the index from above named idx_origin_city
*/

SELECT * FROM sqlite_master WHERE type='index';

 * sqlite:///flight.db
Done.


type,name,tbl_name,rootpage,sql
index,sqlite_autoindex_FLIGHTS_1,FLIGHTS,33677,None
index,idx_origin_city,FLIGHTS,9748,CREATE INDEX idx_origin_city ON FLIGHTS(origin_city)
index,idx_dest_city,FLIGHTS,7472,CREATE INDEX idx_dest_city ON FLIGHTS(dest_city)


Q1.4. (1.5 point) Please check whether each query used the index or not. 

**Hint:** you can use `EXPLAIN QUERY PLAN` to see the query plan of each query. Indicate for each query if it used the index or not. 

In [6]:
%%sql
/*
The index is used by query A
*/

EXPLAIN QUERY PLAN
SELECT DISTINCT carrier_id FROM Flights WHERE origin_city = 'Seattle WA' AND actual_time <= 30;

 * sqlite:///flight.db
Done.


id,parent,notused,detail
4,0,0,SEARCH Flights USING INDEX idx_origin_city (origin_city=?)
18,0,0,USE TEMP B-TREE FOR DISTINCT


In [7]:
%%sql
/*
The index is used by query B
*/

EXPLAIN QUERY PLAN
SELECT DISTINCT carrier_id FROM Flights WHERE origin_city = 'Gunnison CO' AND actual_time <= 180;

 * sqlite:///flight.db
Done.


id,parent,notused,detail
4,0,0,SEARCH Flights USING INDEX idx_origin_city (origin_city=?)
18,0,0,USE TEMP B-TREE FOR DISTINCT


In [9]:
%%sql
/*
The index is used by query C
*/

EXPLAIN QUERY PLAN 
SELECT DISTINCT carrier_id FROM Flights WHERE origin_city = 'Seattle WA' AND actual_time <= 30;

 * sqlite:///flight.db
Done.


id,parent,notused,detail
4,0,0,SEARCH Flights USING INDEX idx_origin_city (origin_city=?)
18,0,0,USE TEMP B-TREE FOR DISTINCT


Now, consider this query:

```sqlite
(d) SELECT DISTINCT F2.origin_city
     FROM Flights F1, Flights F2
     WHERE F1.dest_city = F2.dest_city
         AND F1.origin_city='Gunnison CO'
         AND F1.actual_time <= 30;
```

Q1.5. (2 points) Choose one simple index (index on one attribute), different from the index for the question above, that is likely to speed up this query. Write down the `CREATE INDEX` statement.

In [9]:
%%sql
CREATE INDEX idx_dest_city ON FLIGHTS(dest_city);

 * sqlite:///flight.db
Done.


[]

Check whether the FLIGHTS table has this second index that you indicate above. If not, add this index to the FLIGHTS table. 

In [13]:
%%sql
/*
The FLIGHTS table does indeed have the second index named "idx_dest_city"
*/

SELECT * FROM sqlite_master WHERE type='index';

 * sqlite:///flight.db
Done.


type,name,tbl_name,rootpage,sql
index,sqlite_autoindex_FLIGHTS_1,FLIGHTS,33677,None
index,idx_origin_city,FLIGHTS,9748,CREATE INDEX idx_origin_city ON FLIGHTS(origin_city)
index,idx_dest_city,FLIGHTS,7472,CREATE INDEX idx_dest_city ON FLIGHTS(dest_city)


Now we want to know how effective the two indexes are. We compare the runtimes of the queries with and without indexes. 

**Hint:** Use `timer on` on sqlite3 command line to turn SQL timer on.

Q1.6. (2 points) Execute queries (a) to (d) on the FLIGHTS table that do not have the two indexes. Please record the runtime of each query. 

a) 0.135s b) 0.120s c) 0.121s d) 0.126s

Q1.7. (2 points) Execute queries (a) to (d) on the FLIGHTS table that has the two indexes. Please record the runtime of each query. 

a) 0.02s b) 0.00024s c) 0.022s d) 0.000215s

## Q2 (6 points): B-Trees

Assume:

    (1) blocks can hold either 10 records or 99 keys and 100 pointers
    (2) the average B-tree node is 70% full. This means it will have 69 keys and 70 pointers. 

We can use B-trees as part of several different structures. For each structure described in the questions Q2.1 to Q2.3 below, determine: 

    (a) the total number of blocks needed for a 1,000,000-record file
    (b) the average number of disk I/O’s to retrieve a record given its search key

You may assume nothing is in memory initially, and the search key is the primary key for the records.

Q2.1. (2 points) The data file is a sequential file, sorted on the search key, with 10 records per block. The B-tree is a dense index.


a) 1,000,000/10 = 100,000 for lowest level

1,000,000/69 = 14,493 for leaf level

14,493/70 = 208 for pre-leaf level

208/70 = 3 for second level

1 for the first level

total number of blocks needed are 100,000+14,493+208+3+1 = 114,705

b) The average number of disk I/O's to retrieve a record given its search key is 5, since the tree has 4 levels.

Q2.2. (2 points) The data file is a sequential file, sorted on the search key, with 10 records per block. The B-tree is a sparse index.

a) 1,000,000/10 = 100,000 to hold data file

100,000/69 = 1450 to hold index file on leaf level

1450/70 = 21 to hold index file on leaf level + 1

21/70 = 1 to hold index file on level 1

100000+1450+21+1 = 101,472 blocks

b) since the tree has 4 levels, 5 disk reads are required.

Q2.3. (2 points) The data file consists of records in no particular order, packed 10 to a block. The B-tree is a dense index.

a) since the tree is still a dense index, this is the same problem as 2.1.
Therefore there are 114,705 blocks neeeded.

b) Levels of the tree are the same being 4 so 5 disk reads are required.

## Q3 (9 points): Query Processing

In the first assignment, given the bank database below:

 - Customer = {<span style="text-decoration:underline">customerID</span>, firstName, lastName, income, birthDate}
 - Account = {<span style="text-decoration:underline">accNumber</span>, type, balance, branchNumber<sup>FK-Branch</sup>}
 - Owns = {<span style="text-decoration:underline">customerID</span><sup>FK-Customer</sup>, <span style="text-decoration:underline">accNumber</span><sup>FK-Account</sup>}
 - Transactions = {<span style="text-decoration:underline">transNumber</span>, <span style="text-decoration:underline">accNumber</span><sup>FK-Account</sup>, amount}
 - Employee = {<span style="text-decoration:underline">sin</span>, firstName, lastName, salary, branchNumber<sup>FK-Branch</sup>}
 - Branch = {<span style="text-decoration:underline">branchNumber</span>, branchName, managerSIN<sup>FK-Employee</sup>, budget}

you wrote a SQL query to:

Show account number, account type, account balance, and transaction amount of the accounts with balance higher than 100,000 and transaction amounts higher than 15000, starting with the accounts with the highest transaction amount and highest account balance. 

Q3.1. (3 points) Parse your query into a query parse tree.

 <img src="./3_one.jpg" alt="If image does not load picture is attached under name 3_one.jpg" />

Q3.2. (3 points) Convert your parse tree to the equivalent relational algebraic representation (rewrite if necessary).

<img src="./3_two.jpg" alt="If image does not load picture is attached under name 3_two.jpg" /> 

Q3.3. (3 points) Assume you have a million records in each of the six tables above. If you need, make necessary assumptions about your storage blocks, as well as about charactristics in the bank.db. Can you estimate the size and cost of the intermediate tables in your query plan?

Assumptions:
1. Data Types: Assuming integer data types for IDs and numeric data types for values (e.g., income, balance, salary, amount).
2. Storage Block Size: Assuming a storage block size of 4 KB.
3. Indexing: Assuming some primary keys and foreign keys are indexed for efficient joins.

Size and Cost:
1. Size of the Account and Transactions Tables:
- Each record in the Account table could be around 40 bytes (assuming four integer columns and one character column on average).
- Each record in the Transactions table could be around 24 bytes (assuming two integer columns and one numeric column).
- Since there are a million records in each table, the size of each table would be approximately:
- Account Table: 40 bytes/record * 1,000,000 records = 40,000,000 bytes (40 MB)
- Transactions Table: 24 bytes/record * 1,000,000 records = 24,000,000 bytes (24 MB)

2. Size of the Intermediate Table after Join:
- The intermediate result of joining the Account and Transactions tables would include the columns: accNumber, type, balance, and amount.
- Assuming that each of these columns is represented by an integer or a numeric data type, the size of each record in the intermediate table could be around 16 bytes (4 bytes each for four columns).
- Since there are a million records in the intermediate table, the size of the intermediate result would be approximately:
- Intermediate Table: 16 bytes/record * 1,000,000 records = 16,000,000 bytes (16 MB)

3. Cost of the Query Plan:
- The cost of the query plan depends on various factors, such as the database management system, hardware configuration, indexing, and query optimization.
- The cost of the query is very similar to the size, so the cost is approximately 16,000,000 bytes (16 MB) aswell.


## Q4 (10 points) Normalization

Consider the schema over attributes A, B, C, D, E, F and the following set of FDs:

EF → BC 

A → D 

B → AE 

BD → C


Q4.1 (5 points) Find a lossless BCNF decomposition. Is it dependency-preserving?

 <img src="./4_one.jpg" alt="If image does not load picture is attached under name 4_one.jpg" />

Q4.2 (5 points) Is the schema in 3NF? If not, apply the 3NF synthesis algorithm to obtain a lossless, dependency-
preserving 3NF decomposition.

<img src="4_two.jpg" alt="If image does not load picture is attached under name 4_two.jpg" />

## Submission

Complete the answers to the questions in the [hw4.ipynb](hw4.ipynb) notebook and zip the notebook with additional image files that you may have used in a file named HW4.zip, and submit it through Canvas system to your Homework (4) activity.